## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [3]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [4]:
Tag_client = AxClient()

[INFO 04-04 12:45:24] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [5]:
# Define bounds for each parameter type
int_bounds = {
    "z_s": [1, 15], #NOTE - z_s is actually half z_s (which must be even)
    "z_p2": [25, 35],
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.5, 0.5],
    "x_p1": [-1.0, 1.0],
    "x_p2": [-0.5, 0.5],
    "x_r2": [-1.5, 1.5],
}

# Number of copies for each parameter
num_copies = 3

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.01*(num_copies-1-i)), upper - .01*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        # parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Viable_Cross_Terms": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-04 12:45:24] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_s', parameter_type=INT, range=[3, 15]), RangeParameter(name='z_s_1', parameter_type=INT, range=[2, 14]), RangeParameter(name='z_s_2', parameter_type=INT, range=[1, 13]), RangeParameter(name='z_p2', parameter_type=INT, range=[27, 35]), RangeParameter(name='z_p2_1', parameter_type=INT, range=[26, 34]), RangeParameter(name='z_p2_2', parameter_type=INT, range=[25, 33]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.48, 0.5]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.49, 0.49]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.5, 0.48]), RangeParameter(name='x_p1', parameter_type=FLOAT, range=[-0.98, 1.0]), RangeParameter(name='x_

In [6]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [7]:
parameters

[{'name': 'z_s', 'type': 'range', 'bounds': [3, 15], 'value_type': 'int'},
 {'name': 'z_s_1', 'type': 'range', 'bounds': [2, 14], 'value_type': 'int'},
 {'name': 'z_s_2', 'type': 'range', 'bounds': [1, 13], 'value_type': 'int'},
 {'name': 'z_p2', 'type': 'range', 'bounds': [27, 35], 'value_type': 'int'},
 {'name': 'z_p2_1', 'type': 'range', 'bounds': [26, 34], 'value_type': 'int'},
 {'name': 'z_p2_2', 'type': 'range', 'bounds': [25, 33], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [72, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [71, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [70, 84], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.48, 0.5],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.49, 0.49],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.5, 0.48],
  'value_type': 'float'},
 {'name': 'x_p1',
  'type': 'ran

In [8]:
parameter_constraints

['x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_p1 - x_p1_1 >= 0.1',
 'x_p1_1 - x_p1_2 >= 0.1',
 'x_p2 - x_p2_1 >= 0.1',
 'x_p2_1 - x_p2_2 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1']

### Run tests to solve for best possible score    

In [9]:
parameterization, trial_index = Tag_client.get_next_trial()

[INFO 04-04 12:45:24] ax.service.ax_client: Generated new trial 0 with parameters {'z_s': 4, 'z_s_1': 14, 'z_s_2': 2, 'z_p2': 34, 'z_p2_1': 32, 'z_p2_2': 30, 'z_r2': 85, 'z_r2_1': 79, 'z_r2_2': 80, 'x_s': 0.433086, 'x_s_1': 0.19825, 'x_s_2': -0.267983, 'x_p1': 0.406724, 'x_p1_1': -0.487027, 'x_p1_2': -0.819365, 'x_p2': 0.411736, 'x_p2_1': 0.287, 'x_p2_2': -0.358565, 'x_r2': -0.481922, 'x_r2_1': -0.624526, 'x_r2_2': -1.22582} using model Sobol.


In [10]:
parameterization

{'z_s': 4,
 'z_s_1': 14,
 'z_s_2': 2,
 'z_p2': 34,
 'z_p2_1': 32,
 'z_p2_2': 30,
 'z_r2': 85,
 'z_r2_1': 79,
 'z_r2_2': 80,
 'x_s': 0.43308586079627276,
 'x_s_1': 0.19825046056881546,
 'x_s_2': -0.2679832748509944,
 'x_p1': 0.4067238404043019,
 'x_p1_1': -0.48702703295275573,
 'x_p1_2': -0.8193650492094458,
 'x_p2': 0.4117361406236887,
 'x_p2_1': 0.2870000604726374,
 'x_p2_2': -0.3585646120645106,
 'x_r2': -0.48192237433046103,
 'x_r2_1': -0.6245256020314991,
 'x_r2_2': -1.2258196312375367}

In [11]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

21
z_s: 4
z_s_1: 14
z_s_2: 2
z_p2: 34
z_p2_1: 32
z_p2_2: 30
z_r2: 85
z_r2_1: 79
z_r2_2: 80
x_s: 0.43308586079627276
x_s_1: 0.19825046056881546
x_s_2: -0.2679832748509944
x_p1: 0.4067238404043019
x_p1_1: -0.48702703295275573
x_p1_2: -0.8193650492094458
x_p2: 0.4117361406236887
x_p2_1: 0.2870000604726374
x_p2_2: -0.3585646120645106
x_r2: -0.48192237433046103
x_r2_1: -0.6245256020314991
x_r2_2: -1.2258196312375367


In [12]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-04 12:45:24] ax.service.ax_client: Completed trial 0 with data: {'Viable_Cross_Terms': (23, None)}.


In [25]:
for i in range(20):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-04 12:45:54] ax.service.ax_client: Generated new trial 41 with parameters {'z_s': 6, 'z_s_1': 13, 'z_s_2': 3, 'z_p2': 31, 'z_p2_1': 31, 'z_p2_2': 25, 'z_r2': 79, 'z_r2_1': 77, 'z_r2_2': 81, 'x_s': 0.191184, 'x_s_1': -0.365945, 'x_s_2': -0.488125, 'x_p1': 0.28213, 'x_p1_1': 0.178436, 'x_p1_2': -0.560587, 'x_p2': 0.495316, 'x_p2_1': 0.104164, 'x_p2_2': -0.202649, 'x_r2': 0.228735, 'x_r2_1': -0.900247, 'x_r2_2': -1.236712} using model Sobol.
[INFO 04-04 12:45:54] ax.service.ax_client: Completed trial 41 with data: {'Viable_Cross_Terms': (8, None)}.
[INFO 04-04 12:46:29] ax.service.ax_client: Generated new trial 42 with parameters {'z_s': 4, 'z_s_1': 8, 'z_s_2': 2, 'z_p2': 29, 'z_p2_1': 28, 'z_p2_2': 28, 'z_r2': 74, 'z_r2_1': 79, 'z_r2_2': 82, 'x_s': 0.455514, 'x_s_1': -0.036023, 'x_s_2': -0.491559, 'x_p1': 0.297013, 'x_p1_1': -0.292329, 'x_p1_2': -0.625882, 'x_p2': 0.309758, 'x_p2_1': -0.026184, 'x_p2_2': -0.307108, 'x_r2': 1.259227, 'x_r2_1': 0.55061, 'x_r2_2': -0.414799} using 

In [26]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Viable_Cross_Terms,z_s,z_s_1,z_s_2,z_p2,...,x_s_2,x_p1,x_p1_1,x_p1_2,x_p2,x_p2_1,x_p2_2,x_r2,x_r2_1,x_r2_2
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,23.0,4,14,2,34,...,-0.267983,0.406724,-0.487027,-0.819365,0.411736,0.287000,-0.358565,-0.481922,-0.624526,-1.225820
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,0.0,13,14,3,30,...,-0.424703,-0.096240,-0.748804,-0.956622,-0.089115,-0.216367,-0.359825,1.397608,1.015468,0.619523
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,10.0,15,3,4,30,...,-0.362195,0.393837,-0.278691,-0.743073,0.103719,-0.109155,-0.447294,-0.475565,-0.885141,-1.117790
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,15.0,9,8,8,27,...,-0.436929,0.900059,0.235358,-0.653168,-0.140779,-0.320558,-0.447464,1.113364,-0.140545,-1.117658
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,12.0,4,6,1,31,...,-0.444333,0.652787,0.471753,-0.766865,0.478475,0.350862,-0.386281,0.163649,-1.097523,-1.374402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,56,56_0,COMPLETED,BoTorch,GenerationStep_1,24.0,3,7,8,31,...,-0.455630,0.292615,0.059039,-0.967370,0.176251,0.066390,-0.443964,1.365090,0.830403,-1.468630
57,57,57_0,COMPLETED,BoTorch,GenerationStep_1,18.0,6,3,1,28,...,-0.415170,0.405932,-0.059912,-0.197800,0.289946,0.062773,-0.040842,1.488108,0.748246,0.648246
58,58,58_0,COMPLETED,BoTorch,GenerationStep_1,9.0,3,7,11,31,...,-0.500000,-0.181997,-0.281997,-0.941601,0.434147,0.208217,-0.324625,1.500000,1.400000,-1.402053
59,59,59_0,COMPLETED,BoTorch,GenerationStep_1,17.0,5,3,1,28,...,-0.386869,0.515354,-0.203316,-0.431746,0.313649,-0.024471,-0.127743,1.493929,0.593686,0.316628


In [30]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-04 13:48:51] ax.service.ax_client: Generated new trial 161 with parameters {'z_s': 3, 'z_s_1': 7, 'z_s_2': 5, 'z_p2': 31, 'z_p2_1': 33, 'z_p2_2': 29, 'z_r2': 78, 'z_r2_1': 81, 'z_r2_2': 84, 'x_s': 0.367863, 'x_s_1': 0.151035, 'x_s_2': -0.303731, 'x_p1': 0.208029, 'x_p1_1': -0.228656, 'x_p1_2': -0.972278, 'x_p2': 0.302411, 'x_p2_1': 0.15043, 'x_p2_2': -0.426299, 'x_r2': 1.5, 'x_r2_1': -0.554761, 'x_r2_2': -1.5} using model BoTorch.
[INFO 04-04 13:48:51] ax.service.ax_client: Completed trial 161 with data: {'Viable_Cross_Terms': (17, None)}.
[INFO 04-04 13:49:21] ax.service.ax_client: Generated new trial 162 with parameters {'z_s': 3, 'z_s_1': 5, 'z_s_2': 9, 'z_p2': 32, 'z_p2_1': 28, 'z_p2_2': 29, 'z_r2': 76, 'z_r2_1': 71, 'z_r2_2': 83, 'x_s': 0.309096, 'x_s_1': 0.209096, 'x_s_2': -0.5, 'x_p1': 0.984916, 'x_p1_1': 0.884916, 'x_p1_2': -1.0, 'x_p2': 0.102638, 'x_p2_1': 0.002638, 'x_p2_2': -0.5, 'x_r2': 1.439619, 'x_r2_1': -0.217007, 'x_r2_2': -1.5} using model BoTorch.
[INFO 04-04

In [31]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Viable_Cross_Terms,z_s,z_s_1,z_s_2,z_p2,...,x_s_2,x_p1,x_p1_1,x_p1_2,x_p2,x_p2_1,x_p2_2,x_r2,x_r2_1,x_r2_2
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,23.0,4,14,2,34,...,-0.267983,0.406724,-0.487027,-0.819365,0.411736,0.287000,-0.358565,-0.481922,-0.624526,-1.225820
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,0.0,13,14,3,30,...,-0.424703,-0.096240,-0.748804,-0.956622,-0.089115,-0.216367,-0.359825,1.397608,1.015468,0.619523
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,10.0,15,3,4,30,...,-0.362195,0.393837,-0.278691,-0.743073,0.103719,-0.109155,-0.447294,-0.475565,-0.885141,-1.117790
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,15.0,9,8,8,27,...,-0.436929,0.900059,0.235358,-0.653168,-0.140779,-0.320558,-0.447464,1.113364,-0.140545,-1.117658
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,12.0,4,6,1,31,...,-0.444333,0.652787,0.471753,-0.766865,0.478475,0.350862,-0.386281,0.163649,-1.097523,-1.374402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,256,256_0,COMPLETED,BoTorch,GenerationStep_1,22.0,7,7,6,31,...,-0.384694,0.375767,0.275767,0.175767,0.183298,-0.261355,-0.500000,0.128131,0.028131,-0.566577
257,257,257_0,COMPLETED,BoTorch,GenerationStep_1,31.0,4,6,3,31,...,-0.500000,0.690120,0.590120,-0.980752,0.177364,0.077364,-0.500000,1.187558,0.996683,-0.681695
258,258,258_0,COMPLETED,BoTorch,GenerationStep_1,33.0,10,7,8,31,...,-0.302605,0.238400,0.135905,0.012167,0.247947,-0.317304,-0.470164,0.207764,0.107764,0.007764
259,259,259_0,COMPLETED,BoTorch,GenerationStep_1,36.0,3,4,6,31,...,-0.500000,0.572702,0.472702,-1.000000,0.115094,0.015094,-0.500000,0.534170,0.434170,-1.500000


In [89]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")
print(score_vals(param_to_list(best_parameters)))

z_s: 3
z_s_1: 7
z_s_2: 5
z_p2: 31
z_p2_1: 26
z_p2_2: 30
z_r2: 79
z_r2_1: 77
z_r2_2: 78
x_s: -0.02787721685490907
x_s_1: -0.1278772168549121
x_s_2: -0.49999999999999983
x_p1: 0.5700968845679462
x_p1_1: 0.47009688456794524
x_p1_2: -0.9999999999999996
x_p2: 0.1810951516143359
x_p2_1: 0.08109515161433528
x_p2_2: -0.5
x_r2: 0.6753453606330697
x_r2_1: 0.5753453606330694
x_r2_2: -1.5
62


In [90]:
param_to_list(best_parameters)

[array([ 6, 14, 10]),
 array([31, 26, 30]),
 array([79, 77, 78]),
 array([-0.02787722, -0.12787722, -0.5       ]),
 array([ 0.57009688,  0.47009688, -1.        ]),
 array([ 0.18109515,  0.08109515, -0.5       ]),
 array([ 0.67534536,  0.57534536, -1.5       ])]

In [41]:
import importlib

In [44]:
%load_ext autoreload

In [91]:
z_s = np.array([6, 10, 8])/2
z_p1 = np.array([42, 40, 41])
z_p2 = np.array([32, 31, 30])
z_r2 = np.array([80, 78, 77])
x_s = np.array([0.5, 0.4, 0.3])
x_p1 = np.array([0.8, 0.7, 0.6])
x_p2 = np.array([0.4, 0.3, 0.2])
x_r2 = np.array([1.2, 1.0, 0.8])

Yi_vals = [z_s, z_p2, z_r2, x_s, x_p1, x_p2, x_r2] #list of arrays

In [84]:
%autoreload 2
from utils import list_to_param

In [92]:
Yi_list = list_to_param(Yi_vals)


In [93]:
Tag_client.attach_trial(
        parameters=Yi_list
    )

[INFO 04-04 16:18:41] ax.core.experiment: Attached custom parameterizations [{'z_s': 3, 'z_s_1': 5, 'z_s_2': 4, 'z_p2': 32, 'z_p2_1': 31, 'z_p2_2': 30, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 77, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.8, 'x_p1_1': 0.7, 'x_p1_2': 0.6, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.2, 'x_r2_1': 1.0, 'x_r2_2': 0.8}] as trial 362.


({'z_s': 3,
  'z_s_1': 5,
  'z_s_2': 4,
  'z_p2': 32,
  'z_p2_1': 31,
  'z_p2_2': 30,
  'z_r2': 80,
  'z_r2_1': 78,
  'z_r2_2': 77,
  'x_s': 0.5,
  'x_s_1': 0.4,
  'x_s_2': 0.3,
  'x_p1': 0.8,
  'x_p1_1': 0.7,
  'x_p1_2': 0.6,
  'x_p2': 0.4,
  'x_p2_1': 0.3,
  'x_p2_2': 0.2,
  'x_r2': 1.2,
  'x_r2_1': 1.0,
  'x_r2_2': 0.8},
 362)

In [95]:
Tag_client.complete_trial(trial_index=362, raw_data=score_vals(param_to_list(Yi_list)))

[INFO 04-04 16:18:55] ax.service.ax_client: Completed trial 362 with data: {'Viable_Cross_Terms': (130, None)}.


In [ ]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-04 16:20:03] ax.service.ax_client: Generated new trial 363 with parameters {'z_s': 6, 'z_s_1': 4, 'z_s_2': 4, 'z_p2': 29, 'z_p2_1': 30, 'z_p2_2': 26, 'z_r2': 80, 'z_r2_1': 73, 'z_r2_2': 74, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.858097, 'x_p1_1': 0.758097, 'x_p1_2': 0.658097, 'x_p2': 0.430113, 'x_p2_1': 0.330113, 'x_p2_2': 0.230113, 'x_r2': 1.229612, 'x_r2_1': 1.129612, 'x_r2_2': 0.982468} using model BoTorch.
[INFO 04-04 16:20:03] ax.service.ax_client: Completed trial 363 with data: {'Viable_Cross_Terms': (20, None)}.
[INFO 04-04 16:20:36] ax.service.ax_client: Generated new trial 364 with parameters {'z_s': 3, 'z_s_1': 5, 'z_s_2': 4, 'z_p2': 32, 'z_p2_1': 30, 'z_p2_2': 30, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 76, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.852448, 'x_p1_1': 0.752448, 'x_p1_2': 0.629409, 'x_p2': 0.42224, 'x_p2_1': 0.32224, 'x_p2_2': 0.22224, 'x_r2': 1.232352, 'x_r2_1': 1.041803, 'x_r2_2': 0.778718} using model BoTorch.
[INFO 04-04 16:20:36] ax

In [ ]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))